The CommonDataModel has features to join and filter the CDM tables it contains, which can be used for analysis

In [13]:
import matplotlib.pyplot as plt
import carrot
carrot.params['debug_level'] = 0
import glob
import p

In [14]:
inputs = carrot.tools.load_tsv(glob.glob("temp/1/*.tsv"),dtype=None)
cdm = carrot.cdm.CommonDataModel.load(inputs=inputs,
                                         do_mask_person_id=False,
                                         format_level=1)
cdm['person'].dropna(axis=1)

,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,gender_source_value,gender_source_concept_id
person_id,,,,,,,
996,8507,1963,7,16,1963-07-16 00:00:00.000000,Male,8507
997,8507,1969,7,14,1969-07-14 00:00:00.000000,Male,8507
998,8507,1956,7,17,1956-07-17 00:00:00.000000,Male,8507
999,8507,1960,7,16,1960-07-16 00:00:00.000000,Male,8507
1000,8507,1962,7,16,1962-07-16 00:00:00.000000,Male,8507
...,...,...,...,...,...,...,...
1987,8532,1995,7,8,1995-07-08 00:00:00.000000,Female,8532
1988,8532,1956,7,17,1956-07-17 00:00:00.000000,Female,8532
1989,8532,1944,7,20,1944-07-20 00:00:00.000000,Female,8532


In [15]:
model = cdm.filter({
    'person':{
                'gender_concept_id': lambda x: x == 8507,
                'year_of_birth': lambda x: x < 1970
            },
    'observation':{
                'observation_concept_id':lambda x: x == 4288455,
                  }
     }
    )

df = model['person'].merge(model['observation'],on='person_id')
df

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id_x,...,value_as_string,value_as_concept_id,qualifier_concept_id,unit_concept_id,provider_id_y,visit_occurrence_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value
0,999,8507,1960,7,16,1960-07-16 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11.357798214040226,4288455,NaN,NaN
1,1012,8507,1935,7,23,1935-07-23 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,33.04527479891782,4288455,NaN,NaN
2,1021,8507,1956,7,17,1956-07-17 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.100366588058625,4288455,NaN,NaN
3,1021,8507,1956,7,17,1956-07-17 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,i14.100366588058625,4288455,NaN,NaN
4,1021,8507,1956,7,17,1956-07-17 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,69.98379377754527,4288455,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,1540,8507,1968,7,14,1968-07-14 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.255564942453296,4288455,NaN,NaN
132,1540,8507,1968,7,14,1968-07-14 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.797943588710728,4288455,NaN,NaN
133,1540,8507,1968,7,14,1968-07-14 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,32.809025248054155,4288455,NaN,NaN
134,1545,8507,1955,7,18,1955-07-18 00:00:00.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.863750269708095,4288455,NaN,NaN


In [22]:
pd.to_numeric(df['observation_source_value'], errors='coerce')#.plot.hist()
#plt.xlabel('IgG Level')
#plt.yscale('log')
#plt.show();

NameError: name 'pd' is not defined

In [ ]:
import scipy.stats

In [ ]:
def get_serology_analysis(_filter):
    def serology_analysis(self):
        df = self.filter(_filter)
        age = 2022 - df['year_of_birth']
        igg = df['observation_source_value'].astype(float)
        res = scipy.stats.linregress(x=age,y=igg)
        return res
    return serology_analysis

ana = get_serology_analysis({'person':{
                'gender_concept_id': lambda x: x == 8507,
                'year_of_birth': lambda x: x < 1970
            },
            'observation':{
                'observation_concept_id':lambda x: x == 4288455,
             }
             })
ana

In [ ]:
cdm.run_analysis(ana)

In [ ]:
year_of_birth = [(x,x+20) for x in range(1930,2020,20)]
year_of_birth

In [ ]:
genders = [8507,8532]

In [ ]:
carrot.params['debug_level'] = 2

In [ ]:
from itertools import product

results = {}
for ((year_low,year_high),gender) in product(year_of_birth,genders):
    config = {'person':{
                'gender_concept_id': lambda x,gender=gender: x == gender,
                'year_of_birth': lambda x,year_high=year_high,year_low=year_low: ((x < year_high) & (x > year_low))
            },
            'observation':{
                'observation_concept_id':lambda x : x == 4288455,
            }}
    
    ana = get_serology_analysis(config)
    name = f'age_{year_low}_gender_{gender}'
    cdm.add_analysis(ana,name)
    

In [ ]:
results = cdm.run_analyses()
results

In [ ]:
import numpy as np

In [ ]:
x = np.array([i for i in range(0,500)])
for label,res in sorted(results.items()):
    plt.plot(x, res.intercept + res.slope*x, label=label)
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
config = {'person':{
                'year_of_birth': lambda x: ((x < 2030) & (x > 2010))
            },
            'observation':{
                'observation_concept_id':lambda x : x == 4288455,
            }}
df = cdm.filter(config)
df['observation_source_value'].astype(float).plot.hist()